In [1]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp client

<IPython.core.display.Javascript object>

In [3]:
# export

import os
import math
import time
import httpx
import asyncio
import hashlib
import aiohttp

from pathlib import Path

from will_it_saturate.core import Benchmark

<IPython.core.display.Javascript object>

# Caveats

On macOS increase open file limit with:

```
ulimit -n 2048
```

Before starting the fastAPI Server with:

```
uvicorn will_it_saturate.main:app --reload
```

In [4]:
# export


def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return s, size_name[i]

<IPython.core.display.Javascript object>

In [5]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

benchmark = Benchmark(bandwidth=bandwidth, duration=3)

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]
benchmark.create_rows(file_sizes)

<IPython.core.display.Javascript object>

In [23]:
# export


class BaseClient:
    def __init__(self, benchmark, server, client):
        self.benchmark = benchmark
        self.server = server
        self.client = client

    def check_md5sums(self, benchmark_files, responses):
        md5_lookup = {}
        for response in responses:
            url = str(response.url)
            md5_lookup[url] = hashlib.md5(response.content).hexdigest()

        for bf in benchmark_files:
            assert bf.md5sum == md5_lookup.get(bf.url, "wrong")

    def show_results(self):
        print(
            f"Filesize  Transferred for server {self.server} with client {self.client}"
        )
        for row in self.benchmark.rows:
            file_size, file_unit = convert_size(row.file_size)
            transferred_per_second, transferred_unit = convert_size(
                row.bytes_per_second
            )
            file_string = f"{file_size}{file_unit}"
            transferred_string = f"{transferred_per_second}{transferred_unit}/s"
            print(f"{file_string} {transferred_string:>11s}")

<IPython.core.display.Javascript object>

In [24]:
# export


class HttpxClient(BaseClient):
    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        # httpx breaks on more than 100 parallel connections
        max_connections = min(br.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [25]:
bclient = HttpxClient(benchmark, "fastAPI", "httpx")

<IPython.core.display.Javascript object>

In [26]:
await bclient.run()

<IPython.core.display.Javascript object>

In [27]:
bclient.show_results()

Filesize  Transferred for server fastAPI with client httpx
9.54MB   90.34MB/s
976.56KB   87.31MB/s
97.66KB   38.52MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI + httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI + httpx: 
---------
File size: 9.54MB
59.63MB/s
---------
File size: 976.56KB
48.34MB/s
---------
File size: 97.66KB
11.57MB/s


<IPython.core.display.Javascript object>

In [21]:
print("Filesize  Transferred for ")
file_size, file_unit = "97.66", "KB"
file_string = f"{file_size}{file_unit}"
transferred_per_second, transferred_unit = "11.57", "MB"
transferred_string = f"{transferred_per_second}{transferred_unit}/s"
line = f"{file_string} {transferred_string:>10s}"
print(line)

Filesize  Transferred for 
97.66KB  11.57MB/s


<IPython.core.display.Javascript object>

In [20]:
print("Filesize  Transferred for ")
file_size, file_unit = "976.56", "KB"
file_string = f"{file_size}{file_unit}"
transferred_per_second, transferred_unit = "11.57", "MB"
transferred_string = f"{transferred_per_second}{transferred_unit}/s"
line = f"{file_string} {transferred_string:>10s}"
print(line)

Filesize  Transferred for 
976.56KB  11.57MB/s


<IPython.core.display.Javascript object>

In [10]:
print("Benchmark fastAPI + httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI + httpx: 
---------
File size: 9.54MB
46.68MB/s
---------
File size: 976.56KB
42.95MB/s
---------
File size: 97.66KB
26.27MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx: 
---------
File size: 9.54MB
55.56MB/s
---------
File size: 976.56KB
49.69MB/s
---------
File size: 97.66KB
20.27MB/s


<IPython.core.display.Javascript object>

In [24]:
print("Benchmark Nginx with httpx: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with httpx: 
---------
File size: 9.54MB
62.09MB/s
---------
File size: 976.56KB
74.07MB/s
---------
File size: 97.66KB
30.94MB/s


<IPython.core.display.Javascript object>

# aiohttp

In [15]:
# export


class AioHttpResponse:
    def __init__(self, url, content):
        self.url = url
        self.content = content


class AioHttpClient(BaseClient):
    async def fetch_page(self, session, url):
        async with session.get(url) as response:
            content = await response.read()
            return AioHttpResponse(url, content)

    async def measure_benchmark_row(self, br):
        urls = [bf.url for bf in br.files]
        max_connections = min(br.number_of_connections, 200)
        # max_connections = br.number_of_connections
        conn = aiohttp.TCPConnector(limit=max_connections)
        responses = []
        start = time.perf_counter()
        async with aiohttp.ClientSession(connector=conn) as session:
            tasks = [asyncio.create_task(self.fetch_page(session, url)) for url in urls]
            responses = await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        self.responses = responses
        self.check_md5sums(br.files, responses)
        br.elapsed = elapsed

    async def run(self):
        for br in self.benchmark.rows:
            await self.measure_benchmark_row(br)

<IPython.core.display.Javascript object>

In [18]:
bclient = AioHttpClient(benchmark)

<IPython.core.display.Javascript object>

In [19]:
await bclient.run()

<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark fastAPI with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

61.16 MB
54.44 MB
31.9 MB


<IPython.core.display.Javascript object>

In [14]:
print("Benchmark fastAPI with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark fastAPI with aiohttp: 
---------
File size: 9.54MB
49.37MB/s
---------
File size: 976.56KB
52.43MB/s
---------
File size: 97.66KB
35.38MB/s


<IPython.core.display.Javascript object>

In [ ]:
print("Benchmark Nginx with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with aiohttp: 
---------
File size: 9.54MB
60.81MB/s
---------
File size: 976.56KB
56.63MB/s
---------
File size: 97.66KB
40.02MB/s


<IPython.core.display.Javascript object>

In [20]:
print("Benchmark Nginx with aiohttp: ")
for br in benchmark.rows:
    print("---------")
    file_size, file_unit = convert_size(br.file_size)
    print(f"File size: {file_size}{file_unit}")
    file_size, file_unit = convert_size(br.file_size)
    transferred_per_second, transferred_unit = convert_size(br.bytes_per_second)
    print(f"{transferred_per_second}{transferred_unit}/s")

Benchmark Nginx with aiohttp: 
---------
File size: 9.54MB
80.27MB/s
---------
File size: 976.56KB
71.1MB/s
---------
File size: 97.66KB
56.49MB/s


<IPython.core.display.Javascript object>

# Export

In [ ]:
from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_serve_files.ipynb.
Converted 02_run_benchmark.ipynb.
Converted 03_create_files_old.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>